In [2]:
from engine import Value

a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)
d = a*b + c
d._prev

{Value(data=-6.0), Value(data=10.0)}

In [3]:
d._op

'+'

In [ ]:
o.grad = 1.0

# Backpropagation, 使用拓扑序列实现顺序不变性 
# 如果想要更透彻的了解全过程，可以可视化过程每个节点
topo = []
visited = set()
def build_topo(v):
    if v not in visited:
        visited.add(v)
        for child in v._prev:
            build_topo(child)
        topo.append(v)
build_topo(d)

for node in reversed(topo):
    node._backward()